In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random
import warnings
warnings.filterwarnings('ignore')

import matplotlib.image as img

#Import Necessary Libraries
from os import listdir
from pickle import dump, load
import pickle 

from tqdm.notebook import tqdm

from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import L1L2

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add, concatenate

In [2]:
image_directory='../../../data/explore/image_caption_generator/Image_captioning_Dataset/Flicker8k_Dataset/'
img_caption_pkl=pd.read_pickle('../../../data/explore/image_caption_generator/set_0.pkl')

In [70]:

img_names, img_captionIds, img_captions=[], [], []
for row in img_caption_pkl:
    cnt_=sum([1  if char =='#' else 0 for char in row ])
    if cnt_>1:
        print(row, cnt_)
    row=row.split('#')
    img_name, img_caption=row[0], row[1]
    captionId, caption = img_caption.split('\t')
    img_names.append(img_name.split('.jpg')[0])
    img_captionIds.append(captionId)
    img_captions.append(caption)

image_captions=pd.DataFrame({'image':img_names, 'captionId':img_captionIds, 'caption':img_captions})
print(image_captions.shape)
image_captions.head(2)

2837799692_2f1c50722a.jpg#4	Closeup of football player # 25 . 2
3273625566_2454f1556b.jpg#4	A race dog have a muzzle and be wear striped jersey # 8 . 2
2837799692_2f1c50722a.jpg#3	a university of Miami football player # 25 . 2
3147913471_322ea231d9.jpg#2	Florida man 's basketball player # 33 shoot basketball . 2
2833582518_074bef3ed6.jpg#4	A # 2 greyhound dog be run around a track . 2
2769605231_dae8b30201.jpg#1	A black greyhound , with a green and white jersey ( # 7 ) be run on a track . 2
(25000, 3)


,image,captionId,caption
0,318667317_108c402140,4,A man in a black hoodie be hold a paper sign .
1,2072574835_febf0c5fb9,4,Three race dog be run out of the start gate on...


In [71]:
image_captions[image_captions['image']=='319847657_2c40e14113'].head()

,image,captionId,caption
9948,319847657_2c40e14113,3,A woman be tug on a white sheet and laugh
16427,319847657_2c40e14113,1,A girl in a purple shirt be have pull someone ...


In [72]:
image_captions.drop_duplicates().shape

(25000, 3)

In [73]:
image_captions.captionId.nunique(), image_captions.image.nunique(), image_captions.caption.nunique()

(5, 8037, 24817)

In [44]:
# # image_directory='Image_captioning_Dataset/Flicker8k_Dataset'
# for image in random.sample(list(image_captions.image), 3):
#     plt.figure()
#     print(image_captions[image_captions['image']==image].caption.iloc[0])
#     im = img.imread(image_directory + '/'+ image + '.jpg')
#     plt.imshow(im) 
#     plt.show()


In [15]:

def image_feature_extraction(image_directory, model):
    '''
        extract image features from the given directory using pretrained model
    '''
    model.layers.pop()
    model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
    # summarize
    print(model.summary())
    # extract features from each photo
    features = dict()
    for name in tqdm(listdir(image_directory)):
        # load an image from file
        filename = image_directory + '/' + name
        image = load_img(filename, target_size=(224, 224))
        # convert the image pixels to a numpy array
        image = img_to_array(image)
        # reshape data for the model
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # prepare the image for the given model
        image = preprocess_input(image)
        # get features
        feature = model.predict(image, verbose=0)
        # get image id
        image_id = name.split('.')[0]
        # store feature
        features[image_id] = feature
#         print('>%s' % name)
    return features

In [16]:
# # model=ResNet50()
# resnet50=load_model('../../../data/explore/image_caption_generator/resnet50.h5')
# image_features = image_feature_extraction(image_directory, resnet50)
# print('Extracted Features: %d' % len(image_features))
# # save to file
# dump(image_features, open('../../../data/explore/image_caption_generator/output/image_features_reset50.pkl', 'wb'))

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________


Extracted Features: 8091


In [31]:
# load features from pickle
with open(os.path.join('../../../data/explore/image_caption_generator/output/image_features_reset50.pkl'), 'rb') as f:
    image_features = load(f)

In [32]:
len(image_features), image_captions.image.nunique()

(8091, 8037)

In [74]:
image_captions=image_captions[image_captions['image'].isin(image_features.keys())]
print(len(image_features), image_captions.image.nunique(), image_captions.shape)

8091 8036 (24996, 3)


In [75]:
def caption_preprocessing(caption):
    # preprocessing steps
    # convert to lowercase, delete digits, special chars & additional spaces
    caption = caption.lower().replace('[^A-Za-z]', '').replace('\s+', ' ')
    # add start and end tags to the caption
    caption = 'startseq ' + " ".join([word for word in caption.split() if len(word)>1]) + ' endseq'
    return caption
image_captions['caption']=image_captions['caption'].apply(caption_preprocessing)
image_captions['image_feature']=image_captions['image'].apply(lambda x: image_features[x][0])
image_captions.head()

,image,captionId,caption,image_feature
0,318667317_108c402140,4,startseq man in black hoodie be hold paper sig...,"[0.34249333, 0.36296466, 0.002094434, 0.805999..."
1,2072574835_febf0c5fb9,4,startseq three race dog be run out of the star...,"[0.34385532, 0.19550037, 0.6508237, 2.041569, ..."
2,3083016677_5782bc337c,4,startseq two motorcycle with two rider each en...,"[1.0154637, 4.204777, 0.43433255, 0.04175122, ..."
3,95734038_2ab5783da7,0,startseq man on bike nest to river endseq,"[0.67103887, 0.19373809, 0.27402854, 0.0329701..."
4,241346146_f27759296d,1,startseq football player try tackle another pl...,"[0.11336415, 2.6156332, 0.0, 1.4476278, 0.0714..."


In [76]:
image_captions_all=image_captions['caption'].values
print(len(image_captions_all))
image_captions_all[0:5]

24996


array(['startseq man in black hoodie be hold paper sign endseq',
       'startseq three race dog be run out of the start gate on track endseq',
       'startseq two motorcycle with two rider each endseq',
       'startseq man on bike nest to river endseq',
       'startseq football player try tackle another player who have the ball endseq'],
      dtype=object)

In [77]:
# tokenize the text
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(image_captions_all)
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

5340


In [78]:
# get maximum length of the caption available
max_len_caption = max([len(caption.split()) for caption in image_captions_all])
max_len_caption

31

In [79]:
image_captions.head(2)

,image,captionId,caption,image_feature
0,318667317_108c402140,4,startseq man in black hoodie be hold paper sig...,"[0.34249333, 0.36296466, 0.002094434, 0.805999..."
1,2072574835_febf0c5fb9,4,startseq three race dog be run out of the star...,"[0.34385532, 0.19550037, 0.6508237, 2.041569, ..."


In [80]:
image_captions[image_captions['image_feature'].isnull()].shape

(0, 4)

In [81]:
image_captions['caption_token']=image_captions['caption'].apply(lambda x: tokenizer.texts_to_sequences([x])[0])

In [82]:
image_captions.image.nunique()

8036

In [83]:
image_captions.head(2)

,image,captionId,caption,image_feature,caption_token
0,318667317_108c402140,4,startseq man in black hoodie be hold paper sig...,"[0.34249333, 0.36296466, 0.002094434, 0.805999...","[1, 9, 3, 18, 795, 4, 38, 430, 188, 2]"
1,2072574835_febf0c5fb9,4,startseq three race dog be run out of the star...,"[0.34385532, 0.19550037, 0.6508237, 2.041569, ...","[1, 49, 101, 7, 4, 20, 88, 11, 6, 985, 796, 5,..."


In [84]:
len(image_captions['caption_token'].iloc[0])

10

In [85]:
image_names = image_captions['image'].unique()
index = int(len(image_names) * 0.80)
image_names_train=image_names[:index]
image_names_test=image_names[index:]
image_captions_train = image_captions[image_captions['image'].isin(image_names_train)]
image_captions_test = image_captions[image_captions['image'].isin(image_names_test)]
print(image_captions_train.shape, image_captions_test.shape)

(21000, 5) (3996, 5)


In [86]:
image_captions_train.head(2)

,image,captionId,caption,image_feature,caption_token
0,318667317_108c402140,4,startseq man in black hoodie be hold paper sig...,"[0.34249333, 0.36296466, 0.002094434, 0.805999...","[1, 9, 3, 18, 795, 4, 38, 430, 188, 2]"
1,2072574835_febf0c5fb9,4,startseq three race dog be run out of the star...,"[0.34385532, 0.19550037, 0.6508237, 2.041569, ...","[1, 49, 101, 7, 4, 20, 88, 11, 6, 985, 796, 5,..."


In [87]:
image_captions.head(1).image_feature

0    [0.34249333, 0.36296466, 0.002094434, 0.805999...
Name: image_feature, dtype: object

In [89]:
dtexts=image_captions.caption_token.values
dimages=image_captions.image_feature.values

In [90]:
len(dimages[0])

2048

In [91]:
maxlen = np.max([len(text) for text in dtexts])
minlen = np.min([len(text) for text in dtexts])
print("Min length of caption: {} and Max length of caption: {}".format(minlen, maxlen))

Min length of caption: 2 and Max length of caption: 31


In [93]:
def preprocessing(dtexts,dimages):
    N = len(dtexts)
    print("# captions/images = {}".format(N))

    assert(N==len(dimages)) # using assert to make sure that length of images and captions are always similar
    Xtext, Ximage, ytext = [],[],[]
    for text,image in zip(dtexts,dimages):
        # zip() is used to create a tuple of iteratable items
        for i in range(1,len(text)):
            in_text, out_text = text[:i], text[i]
            in_text = pad_sequences([in_text],maxlen=maxlen).flatten()# using pad sequence to make the length of all captions equal
            out_text = to_categorical(out_text,num_classes = vocab_size) # using to_categorical to 

            
            Xtext.append(in_text)
            Ximage.append(image)
            ytext.append(out_text)

    Xtext  = np.array(Xtext)
    Ximage = np.array(Ximage)
    ytext  = np.array(ytext)
    print(" {} {} {}".format(Xtext.shape,Ximage.shape,ytext.shape))
    return(Xtext,Ximage,ytext)


Xtext_train, Ximage_train, ytext_train = preprocessing(image_captions_train.caption_token.values,image_captions_train.image_feature.values)
Xtext_val,   Ximage_val,   ytext_val   = preprocessing(image_captions_test.caption_token.values,image_captions_test.image_feature.values)
# pre-processing is not necessary for testing data
#Xtext_test,  Ximage_test,  ytext_test  = preprocessing(dt_test,di_test)


# captions/images = 21000
 (211644, 31) (211644, 2048) (211644, 5340)
# captions/images = 3996
 (40152, 31) (40152, 2048) (40152, 5340)


In [94]:
from keras import layers
from keras.layers import Input, Flatten, Dropout, Activation
print(vocab_size)

dim_embedding = 64

input_image = Input(shape=(Ximage_train.shape[1],))
fimage = Dense(256, activation='relu', name="ImageFeature")(input_image)
## sequence model
input_txt = Input(shape=(maxlen,))
ftxt = Embedding(vocab_size, dim_embedding, mask_zero=True)(input_txt)
ftxt = LSTM(256, name="CaptionFeature", return_sequences=True)(ftxt)

se2 = Dropout(0.04)(ftxt)
ftxt = LSTM(256, name="CaptionFeature2")(se2)
# combining model for decoder

decoder = add([ftxt, fimage])
decoder = Dense(256, activation="relu")(decoder)
output = Dense(vocab_size, activation='softmax')(decoder)
model = Model(inputs=[input_image, input_txt], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam')

model.summary()

5340
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 31)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 31, 64)       341760      input_2[0][0]                    
__________________________________________________________________________________________________
CaptionFeature (LSTM)           (None, 31, 256)      328704      embedding[0][0]                  
__________________________________________________________________________________________________
dropout (Dropout)               (None, 31, 256)      0           CaptionFeature[0][0]             
__________________________________________________________________________________

In [ ]:
# fit model
from time import time
from keras.callbacks import TensorBoard

# tensorboard = TensorBoard(log_dir="log/{}".format(time()))

hist = model.fit([Ximage_train, Xtext_train], ytext_train, epochs=6, verbose=2, batch_size=32, validation_data=([Ximage_val, Xtext_val], ytext_val))

Epoch 1/6


In [ ]:
for label in ["loss","val_loss"]:
    plt.plot(hist.history[label], label=label)
plt.legend()
plt.xlabel("epochs")
plt.ylabel("loss")
plt.show()

In [137]:
# create data generator to get data in batch (avoids session crash)
def data_generator(image_captions, image_features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    Xseq, Ximage, yseq = list(), list(), list()
    for image in image_captions.image.unique():
        captions = image_captions[image_captions['image']==image].caption.unique()
        # process each caption
        for caption in captions:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([caption])[0]
            # split the sequence into X, y pairs
            for i in range(1, len(seq)):
                # split into input and output pairs
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                # store the sequences
                Xseq.append(image_features[image][0])
                Ximage.append(in_seq)
                yseq.append(out_seq)
    return [Xseq, Ximage], yseq

In [130]:
# data_generator(image_captions.head(1), image_features, tokenizer, max_len_caption, vocab_size, batch_size)

In [120]:
image_captions.head(1)

,image,captionId,caption
0,318667317_108c402140,4,startseq man in black hoodie be hold paper sig...


In [ ]:
image_features

In [160]:
# create data generator to get data in batch (avoids session crash)
def data_generator(image_captions, image_features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    n = 0
    while True:
        for image in image_captions.image.unique():
            n += 1
            captions = image_captions[image_captions['image']==image].caption.unique()
            # process each caption
            for caption in captions:
                # encode the sequence
                seq = tokenizer.texts_to_sequences([caption])[0]
                # split the sequence into X, y pairs
                for i in range(1, len(seq)):
                    # split into input and output pairs
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    
                    # store the sequences
                    X1.append(image_features[image][0])
                    X2.append(in_seq)
                    y.append(out_seq)
            if n == batch_size:
                X1, X2, y = np.array(X1), np.array(X2), np.array(y)
                yield [X1, X2], y
                X1, X2, y = list(), list(), list()
                n = 0

In [164]:
# create data generator to get data in batch (avoids session crash)
def data_generator(image_captions, image_features, tokenizer, max_length, vocab_size, batch_size):
    # loop over images
    X1, X2, y = list(), list(), list()
    
    for image in image_captions.image.unique():
        captions = image_captions[image_captions['image']==image].caption.unique()
        # process each caption
        for caption in captions:
            # encode the sequence
            seq = tokenizer.texts_to_sequences([caption])[0]
            # split the sequence into X, y pairs
            for i in range(1, len(seq)):
                # split into input and output pairs
                in_seq, out_seq = seq[:i], seq[i]
                # pad input sequence
                in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                # encode output sequence
                out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]

                # store the sequences
                X1.append(image_features[image][0])
                X2.append(in_seq)
                y.append(out_seq)
        
    return [X1, X2], y
            

In [165]:
from tensorflow.keras.models import Model
# encoder model
# image feature layers
img_input = Input(shape=(2048,))
img_feature_layer1 = Dropout(0.4)(img_input)
img_feature_layer2 = Dense(128, activation='relu', name="ImageFeature")(img_feature_layer1)
# sequence feature layers
seq_input = Input(shape=(max_len_caption,))
seq_embedding = Embedding(vocab_size, 256, mask_zero=True)(seq_input)
seq_dr_layer = Dropout(0.4)(seq_embedding)
seq_layer1 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_dr_layer)
# seq_layer2 = LSTM(32, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer1)
# seq_layer3 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer2)
# seq_layer4 = LSTM(64, return_sequences=True, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer3)
seq_layer5 = LSTM(128, bias_regularizer=L1L2(l1=.01, l2=0))(seq_layer1)

# decoder model
decoder1 = add([img_feature_layer2, seq_layer5])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[img_input, seq_input], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# plot the model
# plot_model(model, show_shapes=True)

In [104]:
# model.summary()

In [162]:
generator_test = data_generator(image_captions_test.head(2), image_features, tokenizer, max_len_caption, vocab_size, batch_size)
Xtext_Ximage_test, ytext_test=generator_test

generator_train = data_generator(image_captions_train.head(2), image_features, tokenizer, max_len_caption, vocab_size, batch_size)
Xtext_Ximage_train, ytext_train=generator_train

ValueError: too many values to unpack (expected 2)

In [151]:
len(ytext_train[0])

5340

In [ ]:
epochs = 2
batch_size = 32
steps = len(image_names_train) // batch_size

# create train data generator
# generator_train = data_generator(image_captions_train, image_features, tokenizer, max_len_caption, vocab_size, batch_size)
# create train data generator
generator_validation = data_generator(image_captions_test, image_features, tokenizer, max_len_caption, vocab_size, batch_size)
# fit for one epoch
for epoch in range(5)
model_history=model.fit(generator_validation, epochs=epoch, steps_per_epoch=steps)
    
# model_history=model.fit(Xtext_Ximage_train, ytext_train, epochs=6, verbose=2, batch_size=32, validation_data=(Xtext_Ximage_test, ytext_test))
# save model
# model.save('model_' + str(i) + '.h5')

In [ ]:
plt.figure(figsize=(20,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [50]:
len(image_features['318667317_108c402140.jpg'][0])

1000